In [0]:
%pip install -U -qqqq unitycatalog-ai[databricks] mlflow-skinny[databricks] langgraph==0.3.4 databricks-langchain databricks-agents uv
%restart_python

In [0]:
import sys
sys.path.append(".")

from workshop_config import *

In [0]:
%sql
create schema if not exists identifier(:WORKSHOP_CATALOG || "." || :WORKSHOP_SCHEMA);
create volume if not exists identifier(:WORKSHOP_CATALOG || "." || :WORKSHOP_SCHEMA || ".data");
use identifier(:WORKSHOP_CATALOG || "." || :WORKSHOP_SCHEMA);

In [0]:
spark.sql(
    f"grant use schema, select, execute, read volume on schema {WORKSHOP_CATALOG}.{WORKSHOP_SCHEMA} to `account users`"
)

In [0]:
%sh
rm /Volumes/$WORKSHOP_CATALOG/$WORKSHOP_SCHEMA/data/*.csv
cp data/*.csv /Volumes/$WORKSHOP_CATALOG/$WORKSHOP_SCHEMA/data

In [0]:
%sql
create or replace table cust_service_data
select
  * except (_rescued_data)
from
  read_files(
    "/Volumes/" || :WORKSHOP_CATALOG || "/" || :WORKSHOP_SCHEMA || "/data/cust_service_data.csv",
    format => "csv"
  );

create or replace table policies
select
  * except (_rescued_data)
from
  read_files(
    "/Volumes/" || :WORKSHOP_CATALOG || "/" || :WORKSHOP_SCHEMA || "/data/policies.csv",
    format => "csv"
  );

create or replace table product_docs
tblproperties (delta.enableChangeDataFeed = true)
select
  * except (_rescued_data)
from
  read_files(
    "/Volumes/" || :WORKSHOP_CATALOG || "/" || :WORKSHOP_SCHEMA || "/data/product_docs.csv",
    format => "csv",
    multiline => true,
    escape => '"'
  );

In [0]:
%sql
create or replace function get_latest_return()
  returns table(purchase_date date, issue_category string, issue_description string, name string)
  comment 'Returns the most recent customer service interaction, such as returns.'
  return
    (
      select
        cast(date_time as date) as purchase_date,
        issue_category,
        issue_description,
        name
      from
        cust_service_data
      order by
        date_time desc
      limit 1
    );

create or replace function get_return_policy()
  returns table(policy string, policy_details string, last_updated date)
  comment 'Returns the details of the Return Policy'
  language sql
  return
    (
      select
        policy,
        policy_details,
        last_updated
      from
        policies
      where
        policy = 'Return Policy'
      limit 1
    );

create or replace function get_user_id(user_name string)
  returns string
  comment 'This takes the name of a customer as an input and returns the corresponding user_id'
  language sql
  return
    select
      customer_id
    from
      cust_service_data
    where
      name = user_name
    limit 1;

create or replace function get_order_history(user_id string)
  returns table(returns_last_12_months int, issue_category string)
  comment 'This takes the user_id of a customer as an input and returns the number of returns and the issue category'
  language sql
  return
    select
      count(*) as returns_last_12_months,
      issue_category
    from
      cust_service_data
    where
      customer_id = user_id
    group by
      issue_category;

In [0]:
from unitycatalog.ai.core.databricks import DatabricksFunctionClient


def get_todays_date() -> str:
    """
    Returns today's date in 'YYYY-MM-DD' format.

    Returns:
        str: Today's date in 'YYYY-MM-DD' format.
    """
    from datetime import datetime

    return datetime.now().strftime("%Y-%m-%d")

    from unitycatalog.ai.core.databricks import DatabricksFunctionClient


client = DatabricksFunctionClient()

python_tool_uc_info = client.create_python_function(
    func=get_todays_date, catalog=WORKSHOP_CATALOG, schema=WORKSHOP_SCHEMA, replace=True
)
print(f"Deployed Unity Catalog function name: {python_tool_uc_info.full_name}")

In [0]:
from databricks.vector_search.client import VectorSearchClient

client = VectorSearchClient()

if WORKSHOP_CATALOG not in [v["name"] for v in client.list_endpoints()["endpoints"]]:
    print(f"Creating endpoint {WORKSHOP_CATALOG}")
    client.create_endpoint(name=WORKSHOP_CATALOG, endpoint_type="STANDARD")

client.wait_for_endpoint(WORKSHOP_CATALOG)
print(f"Created endpoint {WORKSHOP_CATALOG}")

index_name = f"{WORKSHOP_CATALOG}.{WORKSHOP_SCHEMA}.product_docs_index"
if index_name not in [
    i["name"] for i in client.list_indexes(WORKSHOP_CATALOG).get("vector_indexes", [])
]:
  print(f"Creating index {index_name}")
  client.create_delta_sync_index_and_wait(
        endpoint_name=WORKSHOP_CATALOG,
        source_table_name=f"{WORKSHOP_CATALOG}.{WORKSHOP_SCHEMA}.product_docs",
        index_name=index_name,
        pipeline_type="TRIGGERED",
        primary_key="product_id",
        embedding_source_column="indexed_doc",
        embedding_model_endpoint_name="databricks-gte-large-en",
    )
print(f"Created index {index_name}")

In [0]:
import os
os.environ["IS_WORKSHOP_SETUP"] = "TRUE"
sys.path.append("02_agent_eval")

import mlflow
from agent import tools, LLM_ENDPOINT_NAME
from databricks_langchain import VectorSearchRetrieverTool
from mlflow.models.resources import DatabricksFunction, DatabricksServingEndpoint
from unitycatalog.ai.langchain.toolkit import UnityCatalogTool

resources = [DatabricksServingEndpoint(endpoint_name=LLM_ENDPOINT_NAME)]
for tool in tools:
    if isinstance(tool, VectorSearchRetrieverTool):
        resources.extend(tool.resources)
    elif isinstance(tool, UnityCatalogTool):
        resources.append(DatabricksFunction(function_name=tool.uc_function_name))

input_example = {
    "messages": [
        {
            "role": "user",
            "content": "What color options are available for the Aria Modern Bookshelf?"
        }
    ]
}

with mlflow.start_run():
    logged_agent_info = mlflow.pyfunc.log_model(
        artifact_path="agent",
        python_model="./02_agent_eval/agent.py",
        input_example=input_example,
        resources=resources,
        extra_pip_requirements=[
            "databricks-connect"
        ]
    )



In [0]:
from databricks.sdk import WorkspaceClient
from databricks import agents

mlflow.set_registry_uri("databricks-uc")
uc_model_name = f"{WORKSHOP_CATALOG}.{WORKSHOP_SCHEMA}.product_agent"

# register the model to UC
uc_registered_model_info = mlflow.register_model(
    model_uri=logged_agent_info.model_uri, name=uc_model_name
)

agents.deploy(
    uc_model_name,
    uc_registered_model_info.version,
    scale_to_zero=True,
    environment_vars={
        "WORKSHOP_CATALOG": WORKSHOP_CATALOG,
        "WORKSHOP_SCHEMA": WORKSHOP_SCHEMA,
        "USER_SCHEMA": USER_SCHEMA,
        "IS_WORKSHOP_SETUP": "TRUE",
    },
    tags={"endpointSource": "Agent Lab"},
)